In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("./hcap.sqlite")

attcases = pd.read_sql_query("SELECT * FROM attorney_cases", conn)

##
## How influential is an attorney, or how much have other attorneys cited the work of the attorney?
##

In [53]:
# Find and attorney with a mediumly-large number of cases
# We'll take the last one for our analysis
attorneys = pd.read_sql_query("SELECT * FROM attorneys", conn)
attcascnt = attcases.groupby("attorney_id").size()

print("\nMean Cases:", attcascnt.agg({"id": 'mean'}))

caserng = attcascnt[(attcascnt > 200) & (attcascnt < 210)]
attorney_id = caserng.index.tolist().pop()
attorney_id


Mean Case: id    7.461461
dtype: float64


34124

In [56]:
## Now lets find the attorney
satt = attorneys[attorneys["id"] == attorney_id]
print(satt.to_latex())
satt

\begin{tabular}{lrl}
\toprule
{} &     id &             names \\
\midrule
34123 &  34124 &  william j. tuohy \\
\bottomrule
\end{tabular}



,id,names
34123,34124,william j. tuohy


In [58]:
## Select cases that cite to any of the cases in which the attorney has participated.
citing_cases = pd.read_sql_query(
"""
SELECT 
	case_id, count(*) AS cited_count
FROM 
	case_citations 
WHERE
	cites_to_case_id IN (
		SELECT case_id FROM attorney_cases WHERE attorney_id = ?
	)
GROUP BY
	case_id
""", conn, params=(attorney_id,))
citecnt = citing_cases.agg({"cited_count": ['sum', 'max', 'mean']})
print(citecnt.to_latex())
citecnt



\begin{tabular}{lr}
\toprule
{} &  cited\_count \\
\midrule
sum  &  1867.000000 \\
max  &     7.000000 \\
mean &     1.186904 \\
\bottomrule
\end{tabular}



,cited_count
sum,1867.000000
max,7.000000
mean,1.186904
